<a href="https://colab.research.google.com/github/Anakha-s/Project-1/blob/main/CovidSenti_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## EDA Pkgs
import pandas as pd
#Data Viz Pkg
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import numpy as np # linear algebra
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Load Dataset
df = pd.read_csv("covid19_tweets.csv")
# Preview
df.head(3)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False


In [ ]:
!pip install neattext
# Load Text Cleaning Package
import neattext.functions as nfx

     |████████████████████████████████| 114 kB 7.5 MB/s 


In [ ]:
# Cleaning Text
df['clean_tweet'] = df['text'].apply(nfx.remove_hashtags)
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: nfx.remove_userhandles(x))
# Cleaning Text: Multiple WhiteSpaces
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_multiple_spaces)
# Cleaning Text : Remove urls
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_urls)
# Cleaning Text: Punctuations
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_puncts)
#Remove blank rows if any.
df['clean_tweet'] .dropna(inplace=True)
# Change all the text to lower case.
df['clean_tweet']  = [entry.lower() for entry in df['clean_tweet'] ]
df[['text','clean_tweet']]

,text,clean_tweet
0,If I smelled the scent of hand sanitizers toda...,if i smelled the scent of hand sanitizers toda...
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,hey and wouldnt it have made more sense to ha...
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,trump never once claimed was a hoax we all cl...
3,@brookbanktv The one gift #COVID19 has give me...,the one gift has give me is an appreciation f...
4,25 July : Media Bulletin on Novel #CoronaVirus...,25 july : media bulletin on novel
...,...,...
179103,Thanks @IamOhmai for nominating me for the @WH...,thanks for nominating me for the challenge i n...
179104,2020! The year of insanity! Lol! #COVID19 http...,2020 the year of insanity lol
179105,@CTVNews A powerful painting by Juan Lucena. I...,a powerful painting by juan lucena its a trib...
179106,"More than 1,200 students test positive for #CO...",more than 1200 students test positive for at m...


In [ ]:
# Tokenization
df['clean_tweet']= [word_tokenize(entry) for entry in df['clean_tweet']]
df['clean_tweet']



0         [if, i, smelled, the, scent, of, hand, sanitiz...
1         [hey, and, wouldnt, it, have, made, more, sens...
2         [trump, never, once, claimed, was, a, hoax, we...
3         [the, one, gift, has, give, me, is, an, apprec...
4                 [25, july, :, media, bulletin, on, novel]
                                ...                        
179103    [thanks, for, nominating, me, for, the, challe...
179104                 [2020, the, year, of, insanity, lol]
179105    [a, powerful, painting, by, juan, lucena, its,...
179106    [more, than, 1200, students, test, positive, f...
179107                     [i, stop, when, i, see, a, stop]
Name: clean_tweet, Length: 179108, dtype: object

In [ ]:
df['clean_tweet'].head(1)

0    [if, i, smelled, the, scent, of, hand, sanitiz...
Name: clean_tweet, dtype: object

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(words):
  return [word for word in words if  not word in stop_words]
df['clean_tweet']=df['clean_tweet'].apply(lambda x: remove_stopwords(x))


def stemming(words):
  ps=PorterStemmer()
  return [ps.stem(word) for word in words]
df['clean_tweet']=df['clean_tweet'].apply(lambda x: stemming(x))

def lemmatizing(words):
    lemmatizer =WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]
df['clean_tweet']=df['clean_tweet'].apply(lambda x: lemmatizing(x))

def final_text(words):
     return ' '.join(words)
df['clean_tweet']=df['clean_tweet'].apply(lambda x:final_text(x))
   


In [ ]:
df['clean_tweet'].head(1)

0    smell scent hand sanit today someon past would...
Name: clean_tweet, dtype: object

In [ ]:
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment_polarity = blob.sentiment.polarity
    sentiment_subjectivity = blob.sentiment.subjectivity
    if sentiment_polarity > 0:
        sentiment_label = 1
    elif sentiment_polarity < 0:
        sentiment_label = -1
    else:
        sentiment_label = 0
    result = {'polarity':sentiment_polarity,
              'subjectivity':sentiment_subjectivity,
              'sentiment':sentiment_label}
    return result

In [ ]:
df['sentiment_results'] = df['clean_tweet'].apply(get_sentiment)
df['sentiment_results']

0         {'polarity': -0.25, 'subjectivity': 0.25, 'sen...
1         {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
2         {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
3         {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
4         {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
                                ...                        
179103    {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
179104    {'polarity': 0.8, 'subjectivity': 0.7, 'sentim...
179105    {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
179106    {'polarity': 0.0625, 'subjectivity': 0.5, 'sen...
179107    {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
Name: sentiment_results, Length: 179108, dtype: object

In [ ]:
df = df.join(pd.json_normalize(df['sentiment_results']))
df.head(3)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,clean_tweet,sentiment_results,polarity,subjectivity,sentiment
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False,smell scent hand sanit today someon past would...,"{'polarity': -0.25, 'subjectivity': 0.25, 'sen...",-0.25,0.25,Negative
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False,hey wouldnt made sen player pay respect a…,"{'polarity': 0.0, 'subjectivity': 0.0, 'sentim...",0.00,0.00,Neutral
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False,trump never claim hoax claim effort to…,"{'polarity': 0.0, 'subjectivity': 0.0, 'sentim...",0.00,0.00,Neutral


In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['clean_tweet'],df['sentiment'],test_size=0.3)

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=10000)
Tfidf_vect.fit(df['clean_tweet'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

print(Tfidf_vect.vocabulary_)

{'smell': 8316, 'scent': 7951, 'hand': 4468, 'sanit': 7899, 'today': 9075, 'someon': 8382, 'past': 6746, 'would': 9876, 'think': 8987, 'that': 8949, 'hey': 4594, 'wouldnt': 9877, 'made': 5702, 'sen': 8046, 'player': 6944, 'pay': 6778, 'respect': 7661, 'trump': 9223, 'never': 6304, 'claim': 2373, 'hoax': 4646, 'effort': 3496, 'to': 9069, 'one': 6512, 'gift': 4248, 'give': 4258, 'appreci': 1370, 'simpl': 8229, 'thing': 8986, 'alway': 1242, 'around': 1405, 'me': 5853, '25': 443, 'juli': 5249, 'medium': 5885, 'bulletin': 2017, 'novel': 6404, 'death': 2991, 'continu': 2675, 'rise': 7752, 'almost': 1219, 'bad': 1579, 'ever': 3692, 'politician': 6999, 'busi': 2040, 'want': 9667, 'chang': 2251, 'work': 9841, 'gener': 4211, 'recruit': 7494, 'specif': 8440, 'via': 9555, 'wear': 9702, 'face': 3795, 'cover': 2781, 'shop': 8164, 'includ': 4895, 'visit': 9607, 'local': 5610, 'commun': 2521, 'pray': 7080, 'good': 4302, 'health': 4538, 'recoveri': 7490, 'pope': 7013, 'god': 4286, 'prophet': 7214, 'wat

In [ ]:
print(Train_X_Tfidf)

  (0, 9075)	0.23314617862095663
  (0, 7974)	0.42919361143511414
  (0, 4650)	0.34656232683369664
  (0, 4152)	0.3284576148828056
  (0, 3148)	0.554139467517114
  (0, 125)	0.47578913678507967
  (1, 8046)	0.3457063656943141
  (1, 7194)	0.360394136796339
  (1, 6748)	0.44466551106962254
  (1, 6688)	0.20785858183540407
  (1, 6232)	0.27631688616845707
  (1, 5702)	0.2910304942705596
  (1, 3926)	0.26975055731345526
  (1, 3024)	0.4566957248735831
  (1, 2496)	0.25939246256954185
  (2, 8482)	0.2946636200125008
  (2, 8298)	0.3675328582586364
  (2, 7675)	0.38015247585572703
  (2, 6816)	0.23496889465220475
  (2, 5352)	0.29498477293361935
  (2, 4574)	0.2727138186847275
  (2, 3697)	0.3433162190806219
  (2, 3435)	0.44604364739358376
  (2, 2496)	0.31647568304651325
  (3, 9667)	0.2760270092185996
  :	:
  (125371, 5555)	0.22023431659006512
  (125371, 5325)	0.28570087505049885
  (125371, 1011)	0.5040044365736391
  (125372, 4532)	0.4061098172813791
  (125372, 3699)	0.4149818958510219
  (125372, 2416)	0.3573761

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  98.11289151917816


In [ ]:

confusion_matrix(Test_Y,predictions_SVM)

array([[ 6443,    86,   227],
       [  152, 31057,   280],
       [  206,    63, 15219]])

In [ ]:
target_names = ['negative -1', 'neutral 0', 'positive 1']
print(classification_report(Test_Y,predictions_SVM,target_names=target_names))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95      6756
           1       1.00      0.99      0.99     31489
           2       0.97      0.98      0.98     15488

    accuracy                           0.98     53733
   macro avg       0.97      0.97      0.97     53733
weighted avg       0.98      0.98      0.98     53733



In [ ]:

from sklearn.metrics import classification_report

In [ ]:
review = """I Love stay at home at this pandemic"""
review_vector = Tfidf_vect.transform([review])
print(SVM.predict(review_vector))

[2]
